In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import folium
import random 
import math
from typing import List
from collections import defaultdict
import os
import pickle
import json
import sys



In [ ]:
data = pd.read_excel(excel_path)
data

,SJ_NUMBER,SJ_LOADING_DATE,CARRIER_CODE,CARRIER_DESCRIPTION,WEIGHT_CAPACITY,VOLUME_CAPACITY,SHIP_TO_ORG_ID,LONGITUDE,LATITUDE,SHIPPED_QUANTITY,TOTAL_WEIGHT,TOTAL_VOLUME
0,SJ-202404071,2024-05-20 10:28:00,CB02,COLT L-300,1160,4.70,3367684,-6.135728,106.790511,24,7.000008,0.008687
1,SJ-202404853,2024-06-14 10:27:21,CB02,COLT L-300,1160,4.70,3367732,-6.144215,106.796802,3,0.699290,0.000980
2,SJ-202403866,2024-05-13 15:59:08,CB01,MOTOR BOX,50,0.14,1167762,-6.167749,106.872207,1,0.143000,0.000620
3,SJ-202403699,2024-05-07 16:26:40,CB01,MOTOR BOX,50,0.14,1167762,-6.167749,106.872207,15,0.541667,0.004983
4,SJ-202405662,2024-07-10 15:36:31,CB01,MOTOR BOX,50,0.14,1167762,-6.167749,106.872207,80,2.066667,0.007146
...,...,...,...,...,...,...,...,...,...,...,...,...
28971,SJ-202405310,2024-06-29 11:01:50,CB02,COLT L-300,1160,4.70,4750208,-6.193153,106.803224,24,7.000008,0.008687
28972,SJ-202405560,2024-07-08 09:37:50,CB03,COLT ENGKEL 3T/7M3,2850,7.68,4751111,-6.304409,106.862343,6,2.810000,0.011513
28973,SJ-202405418,2024-07-03 10:45:39,CB01,MOTOR BOX,50,0.14,4751967,NaN,NaN,6,1.193000,0.003419
28974,SJ-202405777,2024-07-13 13:30:57,CB06,MINI BUS,700,4.30,4758127,NaN,NaN,1212,540.800000,1.723297


In [3]:
data = data.drop(['SJ_NUMBER', 'SHIP_TO_ORG_ID', 'SHIPPED_QUANTITY'], axis=1)
data

,SJ_LOADING_DATE,CARRIER_CODE,CARRIER_DESCRIPTION,WEIGHT_CAPACITY,VOLUME_CAPACITY,LONGITUDE,LATITUDE,TOTAL_WEIGHT,TOTAL_VOLUME
0,2024-05-20 10:28:00,CB02,COLT L-300,1160,4.70,-6.135728,106.790511,7.000008,0.008687
1,2024-06-14 10:27:21,CB02,COLT L-300,1160,4.70,-6.144215,106.796802,0.699290,0.000980
2,2024-05-13 15:59:08,CB01,MOTOR BOX,50,0.14,-6.167749,106.872207,0.143000,0.000620
3,2024-05-07 16:26:40,CB01,MOTOR BOX,50,0.14,-6.167749,106.872207,0.541667,0.004983
4,2024-07-10 15:36:31,CB01,MOTOR BOX,50,0.14,-6.167749,106.872207,2.066667,0.007146
...,...,...,...,...,...,...,...,...,...
28971,2024-06-29 11:01:50,CB02,COLT L-300,1160,4.70,-6.193153,106.803224,7.000008,0.008687
28972,2024-07-08 09:37:50,CB03,COLT ENGKEL 3T/7M3,2850,7.68,-6.304409,106.862343,2.810000,0.011513
28973,2024-07-03 10:45:39,CB01,MOTOR BOX,50,0.14,NaN,NaN,1.193000,0.003419
28974,2024-07-13 13:30:57,CB06,MINI BUS,700,4.30,NaN,NaN,540.800000,1.723297


In [4]:
data = data.dropna()
data.isnull().sum()

SJ_LOADING_DATE        0
CARRIER_CODE           0
CARRIER_DESCRIPTION    0
WEIGHT_CAPACITY        0
VOLUME_CAPACITY        0
LONGITUDE              0
LATITUDE               0
TOTAL_WEIGHT           0
TOTAL_VOLUME           0
dtype: int64

In [5]:
data['SJ_LOADING_DATE'] = pd.to_datetime(data['SJ_LOADING_DATE'])

C:\Users\levina\AppData\Local\Temp\ipykernel_28400\4158394208.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['SJ_LOADING_DATE'] = pd.to_datetime(data['SJ_LOADING_DATE'])


In [ ]:
tanggal = '2024-05-03'
waktu_mulai = '07:00:00'
waktu_akhir = '12:00:00'

start_datetime = pd.to_datetime(f"{tanggal} {waktu_mulai}")
end_datetime = pd.to_datetime(f"{tanggal} {waktu_akhir}")

In [7]:
data_used = data[(data['SJ_LOADING_DATE'] >= start_datetime) & (data['SJ_LOADING_DATE'] <= end_datetime)]
data_used = data_used.drop(['SJ_LOADING_DATE', 'CARRIER_CODE', 'CARRIER_DESCRIPTION', 'WEIGHT_CAPACITY', 'VOLUME_CAPACITY'], axis=1).reset_index(drop=True)

depo_point = pd.DataFrame({'LONGITUDE': [-6.181794196399728], 'LATITUDE': [106.91137272072216], 'TOTAL_WEIGHT': [0] , 'TOTAL_VOLUME': [0]})

data_used = pd.concat([depo_point, data_used], ignore_index=True)
data_used

,LONGITUDE,LATITUDE,TOTAL_WEIGHT,TOTAL_VOLUME
0,-6.181794,106.911373,0.000000,0.000000
1,-6.254061,106.790543,5.888000,0.042966
2,-6.177427,106.894859,0.304944,0.001059
3,-6.137841,106.870198,3.065689,0.003800
4,-6.172385,106.898890,0.504562,0.001938
5,-6.147109,106.907927,1.008000,0.002227
6,-6.158287,106.870624,102.090000,0.438742
7,-6.241767,106.839168,68.503198,0.478626
8,-6.227326,106.804837,111.918233,0.524385


In [8]:
longlat = data_used.drop(['TOTAL_WEIGHT', 'TOTAL_VOLUME'], axis=1)
longlat

,LONGITUDE,LATITUDE
0,-6.181794,106.911373
1,-6.254061,106.790543
2,-6.177427,106.894859
3,-6.137841,106.870198
4,-6.172385,106.898890
5,-6.147109,106.907927
6,-6.158287,106.870624
7,-6.241767,106.839168
8,-6.227326,106.804837


In [9]:
unique_carriers = data.drop_duplicates(subset=['CARRIER_CODE'])

carrier = unique_carriers[['CARRIER_CODE', 'CARRIER_DESCRIPTION', 'WEIGHT_CAPACITY', 'VOLUME_CAPACITY']].reset_index(drop=True)
carrier

,CARRIER_CODE,CARRIER_DESCRIPTION,WEIGHT_CAPACITY,VOLUME_CAPACITY
0,CB02,COLT L-300,1160,4.70
1,CB01,MOTOR BOX,50,0.14
2,CB04,COLT 6 BAN,4000,12.00
3,CB06,MINI BUS,700,4.30
4,CB03,COLT ENGKEL 3T/7M3,2850,7.68
5,BU1445,BUILT UP 14T/45M3,14000,45.00


In [10]:
num_carriers = len(carrier)
num_locations = len(longlat)

def haversine(lon1, lat1, lon2, lat2):
    R = 6371.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi / 2.0)**2 + \
        math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2.0)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def calculate_haversine_distance_matrix(longlat):
    num_locations = len(data_used)
    distance_matrix = np.zeros((num_locations, num_locations))
    for i in range(num_locations):
        for j in range(num_locations):
            if i != j:
                distance = haversine(
                    data_used.loc[i, 'LONGITUDE'],
                    data_used.loc[i, 'LATITUDE'],
                    data_used.loc[j, 'LONGITUDE'],
                    data_used.loc[j, 'LATITUDE']
                )
                distance_matrix[i][j] = distance
            else:
                distance_matrix[i][j] = np.inf
    return distance_matrix

In [11]:
distance_matrix = calculate_haversine_distance_matrix(longlat)
print(distance_matrix)

[[        inf 13.63605217  1.84161796  4.79354103  1.42092145  1.18543182
   4.59427192  8.2589012  11.93689589]
 [13.63605217         inf 11.85927849  9.61514039 12.33174029 13.49993935
   9.42332968  5.42123951  1.80669658]
 [ 1.84161796 11.85927849         inf  3.02550223  0.47692476  1.75266148
   2.76483881  6.53128098 10.13843667]
 [ 4.79354103  9.61514039  3.02550223         inf  3.37984849  4.20587648
   0.66145479  4.80958211  7.81848894]
 [ 1.42092145 12.33174029  0.47692476  3.37984849         inf  1.2951485
   3.1759021   7.00805765 10.60717647]
 [ 1.18543182 13.49993935  1.75266148  4.20587648  1.2951485          inf
   4.16356839  8.2334417  11.75124844]
 [ 4.59427192  9.42332968  2.76483881  0.66145479  3.1759021   4.16356839
          inf  4.41343216  7.64570448]
 [ 8.2589012   5.42123951  6.53128098  4.80958211  7.00805765  8.2334417
   4.41343216         inf  3.84561451]
 [11.93689589  1.80669658 10.13843667  7.81848894 10.60717647 11.75124844
   7.64570448  3.8456145

In [12]:
def pick_first_point(data_used, carrier_used, carrier_code, row):
    carrier_used = carrier_used
    pick_points = {'points': [], 'current_weight': 0, 'current_volume': 0} 

    depo_index = 0
    pick_points['points'].append(depo_index)

    if 0 in data_used.index:
            data_used = data_used.drop(index=0)

    while True:
        if data_used.empty:
            break  

        random_index = random.choice(data_used.index)
        points = data_used.loc[random_index]

        current_weight = pick_points['current_weight']
        current_volume = pick_points['current_volume']

        if (current_weight + points['TOTAL_WEIGHT'] <= row['WEIGHT_CAPACITY'] and 
            current_volume + points['TOTAL_VOLUME'] <= row['VOLUME_CAPACITY']):
            
            pick_points['points'].append(random_index)
            pick_points['current_weight'] += points['TOTAL_WEIGHT']
            pick_points['current_volume'] += points['TOTAL_VOLUME']

            data_used = data_used.drop(random_index)
            break 

    return pick_points


In [13]:
n = len(data_used)

pheromone_matrix = np.empty((n, n), dtype=float)

# Mengisi matriks dengan format "i-j"
for i in range(n):
    for j in range(n):
        pheromone_matrix[i, j] = f"{1}"
        
# Menampilkan matriks
print(pheromone_matrix)

[[1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [14]:
def calculate_probabilities(pheromone_matrix, distance_matrix, current_point, row, pick_first_points, data_used, unvisited_indices):
    pick_points = {
        'points': pick_first_points['points'].copy(),  
        'current_weight': pick_first_points['current_weight'],
        'current_volume': pick_first_points['current_volume']
    }

    probabilities = {}
    alpha = 1
    beta = 2

    for index in unvisited_indices:
        pheromone_value = float(pheromone_matrix[current_point, index]) ** alpha
        distance_value = (1 / float(distance_matrix[current_point, index] + 1e-10)) ** beta

        total_denominator = 0.0
        for i in unvisited_indices:
            if i != index:
                pheromone_value_i = float(pheromone_matrix[current_point, i])
                distance_value_i = float(distance_matrix[current_point, i])

                if distance_value_i > 0: 
                    total_denominator += pheromone_value_i * (1 / distance_value_i) * beta

        if total_denominator > 0:
            probability = pheromone_value * distance_value / total_denominator
            probabilities[index] = probability

    sorted_keys = sorted(probabilities, key=probabilities.get, reverse=True)

    for key in sorted_keys:
        total_weight = data_used.loc[key, 'TOTAL_WEIGHT']
        total_volume = data_used.loc[key, 'TOTAL_VOLUME']

        if (pick_points['current_weight'] + total_weight <= row['WEIGHT_CAPACITY'] and
            pick_points['current_volume'] + total_volume <= row['VOLUME_CAPACITY']):

            pick_points['points'].append(key)
            pick_points['current_weight'] += total_weight
            pick_points['current_volume'] += total_volume 
            unvisited_indices.remove(key)
            current_point = key
            break

    is_full = (pick_points['current_weight'] >= row['WEIGHT_CAPACITY'] or
                pick_points['current_volume'] >= row['VOLUME_CAPACITY'])   

    return pick_points, current_point, unvisited_indices, is_full


In [15]:
def calculate_total_distance(distance_matrix, points, longlat):
    total_distance = 0
    num_locations = len(longlat)

    # Menghitung total jarak untuk rute yang diambil
    for i in range(len(points) - 1):
        from_index = points[i]
        to_index = points[i + 1]

        if from_index < num_locations and to_index < num_locations:
            total_distance += distance_matrix[from_index, to_index]
        else:
            print(f"Error: Index out of range for {from_index} or {to_index}.")

    return total_distance

In [16]:
# def update_pheromone(pheromone_matrix, ant_paths):
#     rhow = 0.1
#     new_pheromone_matrix = pheromone_matrix.copy()

#     path_count = np.zeros_like(pheromone_matrix)
    
#     for path in ant_paths:
#         for i in range(len(path) - 1):
#             current_point = path[i]
#             next_point = path[i + 1]
#             path_count[current_point, next_point] += 1

#     for i in range(pheromone_matrix.shape[0]):
#         for j in range(pheromone_matrix.shape[1]):
#             if path_count[i, j] > 0:
#                 new_pheromone_matrix[i, j] = (
#                     rhow * pheromone_matrix[i, j] + 1 / path_count[i, j]
#                 )
#             else:
#                 new_pheromone_matrix[i, j] *= rhow

#     return path_count, new_pheromone_matrix


In [17]:
def update_pheromone(pheromone_matrix, distance_matrix, all_routes, Q=1, epsilon=1e-10):
    for route_info in all_routes:
        route = route_info['route']
        
        for i in range(len(route) - 1):
            node_a = route[i]
            node_b = route[i + 1]
            
            distance = distance_matrix[node_a][node_b] + epsilon
            
            pheromone_value = Q / distance
            pheromone_matrix[node_a][node_b] += pheromone_value
            pheromone_matrix[node_b][node_a] += pheromone_value

            pheromone_matrix[node_a][node_b] = round(pheromone_matrix[node_a][node_b], 3)
            pheromone_matrix[node_b][node_a] = round(pheromone_matrix[node_b][node_a], 3)

    return pheromone_matrix


In [18]:
ants = 3
n_iterations = 5
ant_paths = []
final_gbest = float('inf')  
best_ant = None  
best_iteration = None 
best_route_info = None 
rhow = 0.1

for iteration in range(n_iterations):
    print(f"\nIteration {iteration + 1}")
    gbest = float('inf')  
    gbest_ant = None  
    current_best_route_info = None  

    for ant in range(ants):
        lbest = 0  
        all_routes = []  
        global_taken_indices = set()

        carrier_used = carrier.sort_values(by="WEIGHT_CAPACITY", ascending=True)

        for index, row in carrier_used.iterrows():
            carrier_code = row['CARRIER_CODE']

            pick_points = pick_first_point(data_used, carrier_used, carrier_code, row)

            unvisited_indices = data_used.index.difference(global_taken_indices).tolist()

            current_point = pick_points['points'][1] 

            while True:
                previous_points = pick_points['points'].copy()

                pick_points, current_point, unvisited_indices, is_full = calculate_probabilities(
                    pheromone_matrix, distance_matrix, current_point, row, pick_points, 
                    data_used, unvisited_indices
                )

                if previous_points == pick_points['points']:
                    break  

                current_point = pick_points['points'][-1] 

                if current_point in unvisited_indices:
                    unvisited_indices.remove(current_point)

                if is_full or not unvisited_indices:
                    break 

            pick_points['carrier_code'] = carrier_code

            total_distance = calculate_total_distance(distance_matrix, pick_points['points'], longlat)
            lbest += total_distance

            global_taken_indices.update(pick_points['points'])

            all_routes.append({
                'ant': ant + 1,
                'carrier_code': carrier_code,
                'route': pick_points['points'],
                'distance': total_distance
            })
            ant_paths.append(pick_points['points'])  

            if index == 0:
                pheromone_matrix *= rhow

        if lbest < gbest:
            gbest = lbest
            gbest_ant = ant + 1  

            current_best_route_info = [
                {'carrier_code': route['carrier_code'], 
                 'route': route['route'], 
                 'distance': route['distance'],
                 'point_count': len(route['route'])}
                for route in all_routes if route['ant'] == gbest_ant
            ]

        # print(f"Lbest Ant ke-{ant + 1} : {lbest}")

    pheromone_matrix = update_pheromone(pheromone_matrix, distance_matrix, all_routes, Q=1, epsilon=1e-10)
    np.set_printoptions(precision=3, suppress=True)

    print(f"Gbest Iterasi ke-{iteration + 1} : {gbest}")

    if gbest < final_gbest:
        final_gbest = gbest
        best_ant = gbest_ant
        best_iteration = iteration + 1
        best_route_info = current_best_route_info 
        print(f"Gbest Saat ini : {final_gbest} (Ant ke-{best_ant}, Iterasi ke-{best_iteration})")
        # print(f"Route Gbest Saat ini : {best_route_info}")
    else:
        print(f"Gbest Saat ini : {final_gbest} (Ant ke-{best_ant}, Iterasi ke-{best_iteration})")
        # print(f"Route Gbest Saat ini : {best_route_info}")

# Setelah iterasi terakhir, simpan hasil best_route_info ke dalam file untuk digunakan di PHP
with open('best_route_info.pkl', 'wb') as f:
    pickle.dump(best_route_info, f)

print("Best route info saved for session.")



Iteration 1
Gbest Iterasi ke-1 : 45.41133219446568
Gbest Saat ini : 45.41133219446568 (Ant ke-2, Iterasi ke-1)

Iteration 2
Gbest Iterasi ke-2 : 60.22883124653136
Gbest Saat ini : 45.41133219446568 (Ant ke-2, Iterasi ke-1)

Iteration 3
Gbest Iterasi ke-3 : 45.298780075294886
Gbest Saat ini : 45.298780075294886 (Ant ke-1, Iterasi ke-3)

Iteration 4
Gbest Iterasi ke-4 : 54.638923893145126
Gbest Saat ini : 45.298780075294886 (Ant ke-1, Iterasi ke-3)

Iteration 5
Gbest Iterasi ke-5 : 70.1782408499434
Gbest Saat ini : 45.298780075294886 (Ant ke-1, Iterasi ke-3)
Best route info saved for session.


In [19]:
with open("C:/xampp/htdocs/skripsi.com/best_route_info.pkl", "rb") as pkl_file:
    data = pickle.load(pkl_file)

# Menyimpan data ke file .json
with open("C:/xampp/htdocs/skripsi.com/best_route_info.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

print("Data berhasil dikonversi ke JSON.")


Data berhasil dikonversi ke JSON.


In [20]:
# from collections import Counter

# def check_duplicate_points(all_routes):
#     # Mengumpulkan semua titik dari setiap rute carrier
#     all_points = [point for route in all_routes for point in route['route']]
    
#     # Menghitung kemunculan setiap titik
#     point_counts = Counter(all_points)
    
#     # Menyaring titik yang muncul lebih dari satu kali
#     duplicate_points = {point: count for point, count in point_counts.items() if count > 1}
    
#     return duplicate_points

# # Contoh penggunaan setelah `all_routes` terisi
# for iteration in range(n_iterations):
#     # ... (kode utama Anda di sini)

#     # Memanggil fungsi untuk memeriksa duplikasi setelah semua rute untuk setiap ant dihasilkan
#     duplicates = check_duplicate_points(all_routes)
    
#     if duplicates:
#         print(f"Terdapat titik yang sama antar carrier di iterasi ke-{iteration + 1}:")
#         for point, count in duplicates.items():
#             print(f"Titik {point} muncul {count} kali")
#     else:
#         print(f"Tidak ada titik yang sama antar carrier di iterasi ke-{iteration + 1}")


In [21]:
output_dir = "C:/xampp/htdocs/skripsi.com/outputs_aco"
os.makedirs(output_dir, exist_ok=True)  # Buat folder jika belum ada

# Loop untuk setiap carrier dalam best_route_info
for carrier_info in best_route_info:
    carrier_code = carrier_info['carrier_code']
    points = carrier_info['route']
    route_coords = longlat.iloc[points]  # Mendapatkan koordinat berdasarkan rute

    plt.figure(figsize=(10, 6))

    # Plot rute dengan garis dan titik untuk setiap lokasi
    plt.plot(route_coords['LONGITUDE'], route_coords['LATITUDE'], color='black', marker='o', linestyle='-', linewidth=2, markersize=5, label=f'Carrier: {carrier_code}')

    # Plot titik dan label setiap lokasi dalam rute
    for idx, point in enumerate(points):
        row = longlat.iloc[point]
        plt.scatter(row['LONGITUDE'], row['LATITUDE'], color='black', s=50)  
        plt.text(row['LONGITUDE'], row['LATITUDE'], f'{idx + 1}', fontsize=12, ha='right') 

        # Hitung dan tampilkan jarak antar titik
        if idx < len(points) - 1:
            next_idx = idx + 1
            distance = distance_matrix[points[idx], points[next_idx]] 
            
            mid_point_lon = (row['LONGITUDE'] + longlat.iloc[points[next_idx]]['LONGITUDE']) / 2
            mid_point_lat = (row['LATITUDE'] + longlat.iloc[points[next_idx]]['LATITUDE']) / 2
            plt.text(mid_point_lon, mid_point_lat, f'{distance:.2f} km', fontsize=10, ha='center', color='red')

    # Setel batas sumbu dan tampilkan grid
    plt.xlim(longlat['LONGITUDE'].min(), longlat['LONGITUDE'].max())
    plt.ylim(longlat['LATITUDE'].min(), longlat['LATITUDE'].max())

    # Setel judul, label, dan legenda
    plt.title(f'Visualisasi Rute Pengiriman Terbaik: {carrier_code}')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.legend()
    plt.grid()

    # Simpan plot sebagai gambar PNG
    image_path = os.path.join(output_dir, f"best_route_{carrier_code}.png")
    plt.savefig(image_path)
    plt.close()  # Tutup plot setelah disimpan

    print(f"Visualisasi rute terbaik untuk {carrier_code} disimpan di {image_path}")


Visualisasi rute terbaik untuk CB01                 disimpan di C:/xampp/htdocs/skripsi.com/outputs_aco\best_route_CB01                .png
Visualisasi rute terbaik untuk CB06                 disimpan di C:/xampp/htdocs/skripsi.com/outputs_aco\best_route_CB06                .png
Visualisasi rute terbaik untuk CB02                 disimpan di C:/xampp/htdocs/skripsi.com/outputs_aco\best_route_CB02                .png
Visualisasi rute terbaik untuk CB03                 disimpan di C:/xampp/htdocs/skripsi.com/outputs_aco\best_route_CB03                .png
Visualisasi rute terbaik untuk CB04                 disimpan di C:/xampp/htdocs/skripsi.com/outputs_aco\best_route_CB04                .png
Visualisasi rute terbaik untuk BU1445               disimpan di C:/xampp/htdocs/skripsi.com/outputs_aco\best_route_BU1445              .png
